In [1]:
import pandas as pd
import requests
import json
import csv
import time
import datetime
import praw
from pmaw import PushshiftAPI
import praw

In [2]:
reddit = praw.Reddit(client_id='J32hsxMDopeDEPrSFZz5zQ', \
                     client_secret='r8heiWleQYzL7JyKWU7QbwlXFCLmDg', \
                     user_agent='aleksanm', \
                     username='aleksanm', \
                     password='red26!red') # if you want you can use yours credentials
api = PushshiftAPI(praw=reddit)
print(reddit.user.me())

aleksanm


In [3]:
sub='worldnews' 
after = "1598922000" #01.09.2020, 01:00 AM GMT
before = "1598925600" #01.09.2020, 02:00 AM GMT
query = "" # query that can be used to search specific news
subCount = 0
subStats = {}

In [4]:
def getPushshiftData(query, after, before, sub):
    url = 'https://api.pushshift.io/reddit/search/submission/?title='+str(query)+'&size=1000&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    print(url)
    r = requests.get(url)
    data = json.loads(r.text)
    return data['data']

data = getPushshiftData(query, after, before, sub)
print('done!')

https://api.pushshift.io/reddit/search/submission/?title=&size=1000&after=1598922000&before=1598925600&subreddit=worldnews
done!


In [5]:
def collectSubData(subm):
    subData = list()
    title = subm['title']
    url = subm['url']
    try:
        flair = subm['link_flair_text']
    except KeyError:
        flair = "NaN"    
    author = subm['author']
    sub_id = subm['id']
    score = subm['score']

    posts = api.search_submissions(ids=sub_id) # yeah, it's a second request I know, but it is there to fetch 'Score'
                                               # because pushshift doen't provide it
    postDictionary = list(posts)[0]
    # print(f"type of postDictionary is {type(postDictionary)}")
    # print(f"posts.ups!!! is {postDictionary['ups']}")
    score = postDictionary['ups']              # score is number of 'ups'
   
    created = datetime.datetime.fromtimestamp(subm['created_utc']) 
    numComms = subm['num_comments']
    permalink = subm['permalink']
    
    subData.append((sub_id,title,url,author,score,created,numComms,permalink,flair))
    subStats[sub_id] = subData

In [6]:
# Will run until all posts have been gathered 
# from the 'after' date up until before date
while len(data) > 0:
    for submission in data:
        collectSubData(submission)
        subCount+=1
    # Calls getPushshiftData() with the created date of the last submission
    print(len(data))
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    after = data[-1]['created_utc']
    data = getPushshiftData(query, after, before, sub)
    
print(len(data))

Total:: Success Rate: 100.00% - Requests: 1 - Batches: 1 - Items Remaining: 0
Finishing enrichment for 1 items
Total:: Success Rate: 100.00% - Requests: 1 - Batches: 1 - Items Remaining: 0
Finishing enrichment for 1 items
Total:: Success Rate: 100.00% - Requests: 1 - Batches: 1 - Items Remaining: 0
Finishing enrichment for 1 items
Total:: Success Rate: 100.00% - Requests: 1 - Batches: 1 - Items Remaining: 0
Finishing enrichment for 1 items
Total:: Success Rate: 100.00% - Requests: 1 - Batches: 1 - Items Remaining: 0
Finishing enrichment for 1 items
Total:: Success Rate: 100.00% - Requests: 1 - Batches: 1 - Items Remaining: 0
Finishing enrichment for 1 items
Total:: Success Rate: 100.00% - Requests: 1 - Batches: 1 - Items Remaining: 0
Finishing enrichment for 1 items
Total:: Success Rate: 100.00% - Requests: 1 - Batches: 1 - Items Remaining: 0
Finishing enrichment for 1 items
Total:: Success Rate: 100.00% - Requests: 1 - Batches: 1 - Items Remaining: 0
Finishing enrichment for 1 items
T

In [16]:
print(str(len(subStats)) + " submissions have added to list")
print("1st entry is:")
print(list(subStats.values())[0][0][1] + " created: " + str(list(subStats.values())[0][0][5]))
print("Last entry is:")
print(list(subStats.values())[-1][0][1] + " created: " + str(list(subStats.values())[-1][0][5]))

41 submissions have added to list
1st entry is:
If the News Media and Digital Platforms Mandatory Bargaining law is allowed to proceed in Australia Facebook has announced it will block all Australia users from sharing local and international news on Facebook and Instagram created: 2020-09-01 04:01:05
Last entry is:
No ‘ekla cholo’ for him, he believed in ‘sabka saath’ created: 2020-09-01 04:58:34


In [24]:
def updateSubs_file():
    upload_count = 0
    print("input filename of submission file, please add .csv")
    file = "data1.csv"
    with open(file, 'w', newline='', encoding='utf-8') as file: 
        a = csv.writer(file, delimiter=',')
        headers = ["Post ID","Title","Url","Author","Score","Publish Date","Total No. of Comments","Permalink","Flair"]
        a.writerow(headers)
        for sub in subStats:
            a.writerow(subStats[sub][0])
            upload_count+=1
            
        print(str(upload_count) + " submissions have been uploaded")

updateSubs_file()
print("done!")

input filename of submission file, please add .csv
41 submissions have been uploaded
done!


In [31]:
df = pd.read_csv('data1.csv')
df = df.sort_values(by=['Score'], ascending=False)
score_above_2 = df[df["Score"] > 2]
print(score_above_2[["Post ID","Title", "Score", "Publish Date"]])

   Post ID                                              Title  Score  \
24  ikb4oi  Iceland: For The First Time In 3 Weeks, No New...   1070   
7   ikaq8d  Ron Jeremy: adult film star charged with 20 ne...    593   
9   ikaqus  Harry Dunn's parents drop plan to sue US gover...     65   
34  ikbb15  India and China Face Off Again at Border as Tr...     57   
28  ikbaqk  India accuses China of 'provocative military m...     51   
35  ikbcgc  Japanese convenience store chain begins testin...     40   
8   ikaqip  US sanctions could put China's Sri Lanka proje...     37   
6   ikapms  Brazil's Bolsonaro to have kidney stone remove...     17   
18  ikax2n  Reopening: WHO harps on COVID-19 prevention plans      4   
0   ikan1s  If the News Media and Digital Platforms Mandat...      3   

           Publish Date  
24  2020-09-01 04:32:31  
7   2020-09-01 04:06:41  
9   2020-09-01 04:07:48  
34  2020-09-01 04:43:55  
28  2020-09-01 04:43:24  
35  2020-09-01 04:46:30  
8   2020-09-01 04:07:12  

In [21]:
print(df[["Post ID","Title", "Score"]])
print(df[0:20])
# submission = reddit.submission("ikao0d")
# print(f"submission title: {submission.title}")
# print(f"submission score: {submission.score}")

   Post ID                                              Title  Score
24  ikb4oi  Iceland: For The First Time In 3 Weeks, No New...   1070
7   ikaq8d  Ron Jeremy: adult film star charged with 20 ne...    593
9   ikaqus  Harry Dunn's parents drop plan to sue US gover...     65
34  ikbb15  India and China Face Off Again at Border as Tr...     57
28  ikbaqk  India accuses China of 'provocative military m...     51
35  ikbcgc  Japanese convenience store chain begins testin...     40
8   ikaqip  US sanctions could put China's Sri Lanka proje...     37
6   ikapms  Brazil's Bolsonaro to have kidney stone remove...     17
18  ikax2n  Reopening: WHO harps on COVID-19 prevention plans      4
0   ikan1s  If the News Media and Digital Platforms Mandat...      3
36  ikbf2s                              8/31/20: Red and Blue      1
22  ikayfn        The election security hole everyone ignores      1
33  ikbawm  WWII veteran presented Victory Medal decades l...      1
32  ikbawd  Girls selling bracelet